#### 구매행렬표
- 

#### 분석 성능 지표 계산
- 지지도, 신뢰도, 향상도

In [23]:
from itertools import combinations
import pandas as pd
import numpy as np
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

In [3]:
data = {
    'user': [1, 2, 3, 4, 5],
    'movie_list': [
        ['가위손', '다이하드', '블레이드 러너'],
        ['블레이드 러너', '아마데우스', '죽은 시인의 사회'],
        ['가위손', '나 홀로 집에', '죽은 시인의 사회'],
        ['다이하드', '블레이드 러너'],
        ['죽은 시인의 사회', '아마데우스'],
    ]
}

df = pd.DataFrame(data)

In [7]:
# 트랜젝션 데이터(장바구니 분석, 연관 규칙 학습 등의 분야에서 데이터 전처리) 전치리용 도구
te = TransactionEncoder()

# numpy 형태로 반환한다.
te_ary = te.fit(df['movie_list']).transform(df['movie_list'])

df_encoded = pd.DataFrame(te_ary, columns=te.columns_)

print("## 구매 행렬표 ##")
# print(te_ary)
print(df_encoded)

## 구매 행렬표 ##
     가위손  나 홀로 집에   다이하드  블레이드 러너  아마데우스  죽은 시인의 사회
0   True    False   True     True  False      False
1  False    False  False     True   True       True
2   True     True  False    False  False       True
3  False    False   True     True  False      False
4  False    False  False    False   True       True


In [8]:
min_support = 0.4
frequent_itemsets = apriori(df_encoded, min_support=min_support, use_colnames=True)

print("## 빈발 항목 집합 ##")
print(frequent_itemsets)

## 빈발 항목 집합 ##
   support            itemsets
0      0.4               (가위손)
1      0.4              (다이하드)
2      0.6           (블레이드 러너)
3      0.4             (아마데우스)
4      0.6         (죽은 시인의 사회)
5      0.4     (다이하드, 블레이드 러너)
6      0.4  (죽은 시인의 사회, 아마데우스)


In [14]:
rules = association_rules(frequent_itemsets, num_itemsets=len(frequent_itemsets), metric="confidence", min_threshold=0.5)
rules_lift = association_rules(frequent_itemsets, num_itemsets=len(frequent_itemsets), metric="lift", min_threshold=1)

print("## 연관 규칙 (신뢰도 기준) ##")
print(rules)
print("\n")

print("## 연관 규칙 (향상도 기준) ##")
print(rules_lift)
print("\n")

## 연관 규칙 (신뢰도 기준) ##
   antecedents  consequents  antecedent support  consequent support  support  \
0       (다이하드)    (블레이드 러너)                 0.4                 0.6      0.4   
1    (블레이드 러너)       (다이하드)                 0.6                 0.4      0.4   
2  (죽은 시인의 사회)      (아마데우스)                 0.6                 0.4      0.4   
3      (아마데우스)  (죽은 시인의 사회)                 0.4                 0.6      0.4   

   confidence      lift  representativity  leverage  conviction  \
0    1.000000  1.666667               1.0      0.16         inf   
1    0.666667  1.666667               1.0      0.16         1.8   
2    0.666667  1.666667               1.0      0.16         1.8   
3    1.000000  1.666667               1.0      0.16         inf   

   zhangs_metric   jaccard  certainty  kulczynski  
0       0.666667  0.666667   1.000000    0.833333  
1       1.000000  0.666667   0.444444    0.833333  
2       1.000000  0.666667   0.444444    0.833333  
3       0.666667  0.666667   1.000

In [21]:
num_transactions = len(df)

In [22]:
# 지지도 함수
def calc_support(itemset):
    count = 0
    for _, row in df_encoded.iterrows():
        if all(row[item] for item in itemset):
            count += 1
    return count / num_transactions

# 신뢰도 함수
def calc_confidence(antecedent, consequent):
    antecedent_support = calc_support(antecedent)

    if antecedent_support == 0:
        return 0
    joint_support = calc_support(antecedent + consequent)
    return joint_support / antecedent_support

# 향상도 함수
def calc_lift(antecedent, consequent):
    antecedent_support = calc_support(antecedent)
    consequent_support = calc_support(consequent)

    if antecedent_support == 0 or consequent_support == 0:
        return 0
    joint_support = calc_support(antecedent + consequent)
    return joint_support / (antecedent_support + consequent_support)

In [ ]:
min_support = 0
freq_itemsets = []

for i in range(1, 3):
    for itemset in combinations(df_encoded.columns, i):
        support = calc_support(list(itemset))
        if support >= min_support:
            freq_itemsets.append((list(itemset), support))

print("## 빈발 항목 집합 ##")

for itemset, support in freq_itemsets:
    print(f"Itemset: {itemset}, Support: {support:.2f}")

## 빈발 항목 집합 ##
Itemset: ['가위손'], Support: 0.40
Itemset: ['나 홀로 집에'], Support: 0.20
Itemset: ['다이하드'], Support: 0.40
Itemset: ['블레이드 러너'], Support: 0.60
Itemset: ['아마데우스'], Support: 0.40
Itemset: ['죽은 시인의 사회'], Support: 0.60
Itemset: ['가위손', '나 홀로 집에'], Support: 0.20
Itemset: ['가위손', '다이하드'], Support: 0.20
Itemset: ['가위손', '블레이드 러너'], Support: 0.20
Itemset: ['가위손', '아마데우스'], Support: 0.00
Itemset: ['가위손', '죽은 시인의 사회'], Support: 0.20
Itemset: ['나 홀로 집에', '다이하드'], Support: 0.00
Itemset: ['나 홀로 집에', '블레이드 러너'], Support: 0.00
Itemset: ['나 홀로 집에', '아마데우스'], Support: 0.00
Itemset: ['나 홀로 집에', '죽은 시인의 사회'], Support: 0.20
Itemset: ['다이하드', '블레이드 러너'], Support: 0.40
Itemset: ['다이하드', '아마데우스'], Support: 0.00
Itemset: ['다이하드', '죽은 시인의 사회'], Support: 0.00
Itemset: ['블레이드 러너', '아마데우스'], Support: 0.20
Itemset: ['블레이드 러너', '죽은 시인의 사회'], Support: 0.20
Itemset: ['아마데우스', '죽은 시인의 사회'], Support: 0.40


In [29]:
association_rules = []
for itemset, _ in freq_itemsets:
    if len(itemset) >= 2:
        for i in range(1, len(itemset)):
            for antecedent in combinations(itemset, i):
                consequent = list(set(itemset) - set(antecedent))
                confidence = calc_confidence(list(antecedent), consequent)
                lift = calc_lift(list(antecedent), consequent)
                association_rules.append((list(antecedent), consequent, confidence, lift))

print("## 연관 규칙 및 지표 ##")
for antecedent, consequent, confidence, lift in association_rules:
    support = calc_support(antecedent + consequent)
    print(f"규칙: {antecedent} -> {consequent}")
    print(f"  지지도: {support:.2f}")
    print(f"  신뢰도: {confidence:.2f}")
    print(f"  향상도: {lift:.2f}")
    print("-" * 20)

## 연관 규칙 및 지표 ##
규칙: ['가위손'] -> ['나 홀로 집에']
  지지도: 0.20
  신뢰도: 0.50
  향상도: 0.33
--------------------
규칙: ['나 홀로 집에'] -> ['가위손']
  지지도: 0.20
  신뢰도: 1.00
  향상도: 0.33
--------------------
규칙: ['가위손'] -> ['다이하드']
  지지도: 0.20
  신뢰도: 0.50
  향상도: 0.25
--------------------
규칙: ['다이하드'] -> ['가위손']
  지지도: 0.20
  신뢰도: 0.50
  향상도: 0.25
--------------------
규칙: ['가위손'] -> ['블레이드 러너']
  지지도: 0.20
  신뢰도: 0.50
  향상도: 0.20
--------------------
규칙: ['블레이드 러너'] -> ['가위손']
  지지도: 0.20
  신뢰도: 0.33
  향상도: 0.20
--------------------
규칙: ['가위손'] -> ['아마데우스']
  지지도: 0.00
  신뢰도: 0.00
  향상도: 0.00
--------------------
규칙: ['아마데우스'] -> ['가위손']
  지지도: 0.00
  신뢰도: 0.00
  향상도: 0.00
--------------------
규칙: ['가위손'] -> ['죽은 시인의 사회']
  지지도: 0.20
  신뢰도: 0.50
  향상도: 0.20
--------------------
규칙: ['죽은 시인의 사회'] -> ['가위손']
  지지도: 0.20
  신뢰도: 0.33
  향상도: 0.20
--------------------
규칙: ['나 홀로 집에'] -> ['다이하드']
  지지도: 0.00
  신뢰도: 0.00
  향상도: 0.00
--------------------
규칙: ['다이하드'] -> ['나 홀로 집에']
  지지도: 0.00
  신뢰도: 0.00
  향상도: 0.00
